# Лабораторная работа №2
**"Непрерывные выпукло-вогнутые игры"**

**Выполнил: Александров А. Н., ИУ8-104**

**Вариант: 1**

## Задание
Функция ядра имеет вид:

$$H(x, y) = ax^2 + by^2 + cxy + dx + ey,$$

где:

|  a |   b  |   c  |   d  |   e  |
|:--:|:----:|:----:|:----:|:----:|
| -5 | 5/12 | 10/3 | -2/3 | -4/3 |

Найти оптимальные стратегии непрерывной выпукло-вогнутой антагонистической игры аналитическим и численным методами.

In [ ]:
import logging
from pathlib import Path

import numpy as np

from game_theory.utils.matrix_games.brown_robinson.brown_robinson import BrownRobinson
logging.basicConfig(level=logging.INFO, format='%(message)s')